## Load API KEY

In [4]:
from dotenv import load_dotenv, find_dotenv
import os

# Load variables from the nearest .env file (walking up directories if needed)
load_dotenv(find_dotenv(), override=False)

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise RuntimeError("GEMINI_API_KEY not found. Make sure it's set in your .env file.")



## load & split mp3 files

In [7]:
# 讀取 MP3 檔案
from pydub import AudioSegment

input_mp3 = './sound_files/' + 'cut2.mp3'
if not os.path.isfile(file_path):
    raise RuntimeError("MP3檔案不存在")

audio = AudioSegment.from_file(input_mp3)
print(audio)


/home/jack/Documents/working/mm-hackathon-py/.venv/lib/python3.12/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
/home/jack/Documents/working/mm-hackathon-py/.venv/lib/python3.12/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)
/home/jack/Documents/working/mm-hackathon-py/.venv/lib/python3.12/site-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/home/jack/Documents/working/mm-hackathon-py/.venv/lib/python3.12/site-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/ho

FileNotFoundError: [Errno 2] No such file or directory: 'ffprobe'